# Getting Started with Relay

**Welcome to Relay** - the agent governance system that creates an "air gap" between autonomous agents and critical actions.

## What You'll Learn

- ✅ How to protect functions with `@relay.protect()`
- ✅ Understanding manifests and seals
- ✅ Policy-based approvals and denials
- ✅ Viewing the audit trail

## Prerequisites

Make sure Relay infrastructure is running:
```bash
cd ~/relay/infra
docker-compose up -d
```

## 1. Setup and Imports

In [1]:
import sys
sys.path.insert(0, '../')

import requests
from datetime import datetime
from uuid import uuid4
import json

# Check if infrastructure is running
try:
    response = requests.get('http://localhost:8181/health', timeout=2)
    print('✅ OPA is running')
except:
    print('❌ OPA is not running. Start with: cd infra && docker-compose up -d')
    
try:
    import subprocess
    result = subprocess.run(['docker', 'exec', 'relay-postgres', 'pg_isready', '-U', 'relay'],
                          capture_output=True)
    if result.returncode == 0:
        print('✅ PostgreSQL is running')
    else:
        print('⚠️  PostgreSQL may not be ready')
except:
    print('⚠️  Could not check PostgreSQL')

✅ OPA is running
✅ PostgreSQL is running


## 2. Understanding Manifests

A **Manifest** is the core primitive of Relay. It describes an agent action request with full context.

In [2]:
# Create a sample manifest
manifest = {
    "manifest_id": str(uuid4()),
    "timestamp": datetime.utcnow().isoformat(),
    "agent": {
        "agent_id": "getting-started-agent",
        "org_id": "tutorial-org",
        "user_id": "student@example.com"
    },
    "action": {
        "provider": "stripe",
        "method": "create_payment",
        "parameters": {
            "amount": 3500,  # $35.00 in cents
            "currency": "USD",
            "customer_id": "cus_tutorial_001"
        }
    },
    "justification": {
        "reasoning": "Tutorial payment for getting started guide",
        "confidence_score": 1.0
    },
    "environment": "production"
}

print("📋 Manifest Structure:")
print(json.dumps(manifest, indent=2))

📋 Manifest Structure:
{
  "manifest_id": "6d39a383-5e42-4764-a9a8-e2434a811b2f",
  "timestamp": "2026-01-18T04:46:10.559926",
  "agent": {
    "agent_id": "getting-started-agent",
    "org_id": "tutorial-org",
    "user_id": "student@example.com"
  },
  "action": {
    "provider": "stripe",
    "method": "create_payment",
    "parameters": {
      "amount": 3500,
      "currency": "USD",
      "customer_id": "cus_tutorial_001"
    }
  },
  "justification": {
    "reasoning": "Tutorial payment for getting started guide",
    "confidence_score": 1.0
  },
  "environment": "production"
}


/var/folders/f9/c8fh89zs1yg3k4s50rl9vgy00000gn/T/ipykernel_58860/1579825037.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


## 3. Policy Evaluation

Relay uses Open Policy Agent (OPA) to evaluate manifests against policies.

In [3]:
# Send manifest to OPA for evaluation
response = requests.post(
    'http://localhost:8181/v1/data/relay/policies/main',
    json={'input': manifest}
)

result = response.json()['result']

print(f"🛡️  Policy Decision:")
print(f"   Approved: {result.get('allow', False)}")
print(f"   Reason: {result.get('reason', 'N/A')}")
print(f"   Policy Version: {result.get('metadata', {}).get('version', 'N/A')}")

🛡️  Policy Decision:
   Approved: True
   Reason: No policy matched
   Policy Version: 1.0


## 4. Testing Different Amounts

Let's test the $50 spending limit policy with different amounts.

In [4]:
def test_payment(amount, description):
    """Test a payment amount against policies."""
    test_manifest = {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": amount}
        }
    }
    
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': test_manifest}
    )
    
    result = response.json()['result']
    approved = result.get('allow', False)
    
    status = "✅ APPROVED" if approved else "❌ DENIED"
    print(f"{description:30} ${amount/100:6.2f}  →  {status}")
    if not approved:
        print(f"{'':30}         Reason: {result.get('reason', 'N/A')}")
    return approved

print("Testing Payment Amounts:")
print("=" * 70)
test_payment(1000, "Small payment")
test_payment(2500, "Medium payment")
test_payment(4999, "Just under limit")
test_payment(5000, "Exactly at limit")
test_payment(5001, "Just over limit")
test_payment(10000, "Large payment")

Testing Payment Amounts:
Small payment                  $ 10.00  →  ✅ APPROVED
Medium payment                 $ 25.00  →  ✅ APPROVED
Just under limit               $ 49.99  →  ✅ APPROVED
Exactly at limit               $ 50.00  →  ✅ APPROVED
Just over limit                $ 50.01  →  ❌ DENIED
                                       Reason: Payment amount exceeds $50.00 limit
Large payment                  $100.00  →  ❌ DENIED
                                       Reason: Payment amount exceeds $50.00 limit


False

## 5. Real-World Example: Payment Processor

Let's create a simple payment processor that uses Relay for governance.

In [5]:
class PaymentProcessor:
    """Example payment processor with Relay governance."""
    
    def __init__(self, agent_id="payment-processor-001"):
        self.agent_id = agent_id
        self.transactions = []
    
    def process_payment(self, amount, customer_id, description):
        """Process a payment with policy validation."""
        # Build manifest
        manifest = {
            "agent": {
                "agent_id": self.agent_id,
                "org_id": "tutorial-org"
            },
            "action": {
                "provider": "stripe",
                "method": "create_payment",
                "parameters": {
                    "amount": amount,
                    "currency": "USD",
                    "customer_id": customer_id
                }
            },
            "justification": {
                "reasoning": description
            }
        }
        
        # Validate with Relay
        response = requests.post(
            'http://localhost:8181/v1/data/relay/policies/main',
            json={'input': manifest}
        )
        
        result = response.json()['result']
        approved = result.get('allow', False)
        
        if approved:
            # Simulate payment execution
            transaction = {
                'id': f'ch_{len(self.transactions) + 1}',
                'amount': amount,
                'customer': customer_id,
                'status': 'succeeded',
                'timestamp': datetime.now()
            }
            self.transactions.append(transaction)
            print(f"✅ Payment processed: ${amount/100:.2f} for {customer_id}")
            return transaction
        else:
            reason = result.get('reason', 'Policy violation')
            print(f"❌ Payment denied: {reason}")
            raise Exception(f"Payment denied: {reason}")
    
    def get_total(self):
        """Get total processed amount."""
        return sum(t['amount'] for t in self.transactions) / 100

# Create processor and test
processor = PaymentProcessor()

print("🏦 Processing payments...\n")

try:
    processor.process_payment(2500, "cus_001", "Coffee subscription")
    processor.process_payment(4800, "cus_002", "Premium plan upgrade")
    processor.process_payment(6000, "cus_003", "Annual subscription")  # This will fail
except Exception as e:
    print(f"\n⚠️  Transaction blocked by policy\n")

print(f"\n📊 Total processed: ${processor.get_total():.2f}")
print(f"📊 Transactions: {len(processor.transactions)}")

🏦 Processing payments...

✅ Payment processed: $25.00 for cus_001
✅ Payment processed: $48.00 for cus_002
❌ Payment denied: Payment amount exceeds $50.00 limit

⚠️  Transaction blocked by policy


📊 Total processed: $73.00
📊 Transactions: 2


## 6. Visualizing the Flow

Let's trace a complete flow from agent action to policy decision.

In [6]:
def trace_flow(amount):
    """Trace the complete Relay flow."""
    print(f"\n{'='*70}")
    print(f"Tracing flow for ${amount/100:.2f} payment")
    print(f"{'='*70}\n")
    
    # Step 1: Agent prepares action
    print("1️⃣  Agent prepares payment action")
    print(f"   └─ Amount: ${amount/100:.2f}")
    
    # Step 2: Build manifest
    print("\n2️⃣  Build manifest with context")
    manifest = {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": amount}
        }
    }
    print(f"   └─ Provider: stripe")
    print(f"   └─ Method: create_payment")
    
    # Step 3: Send to OPA
    print("\n3️⃣  Send to OPA for policy evaluation")
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': manifest}
    )
    result = response.json()['result']
    
    # Step 4: Policy decision
    print("\n4️⃣  Policy engine evaluates")
    print(f"   └─ Checking: amount ({amount}) <= 5000?")
    print(f"   └─ Result: {amount <= 5000}")
    
    # Step 5: Decision
    approved = result.get('allow', False)
    print("\n5️⃣  Final decision")
    if approved:
        print("   ✅ APPROVED - Action may proceed")
        print("   └─ Agent executes payment")
        print("   └─ Result logged in audit trail")
    else:
        reason = result.get('reason', 'N/A')
        print(f"   ❌ DENIED - Action blocked")
        print(f"   └─ Reason: {reason}")
        print("   └─ Denial logged in audit trail")
    
    return approved

# Trace both approved and denied flows
trace_flow(3000)  # Approved
trace_flow(7000)  # Denied


Tracing flow for $30.00 payment

1️⃣  Agent prepares payment action
   └─ Amount: $30.00

2️⃣  Build manifest with context
   └─ Provider: stripe
   └─ Method: create_payment

3️⃣  Send to OPA for policy evaluation

4️⃣  Policy engine evaluates
   └─ Checking: amount (3000) <= 5000?
   └─ Result: True

5️⃣  Final decision
   ✅ APPROVED - Action may proceed
   └─ Agent executes payment
   └─ Result logged in audit trail

Tracing flow for $70.00 payment

1️⃣  Agent prepares payment action
   └─ Amount: $70.00

2️⃣  Build manifest with context
   └─ Provider: stripe
   └─ Method: create_payment

3️⃣  Send to OPA for policy evaluation

4️⃣  Policy engine evaluates
   └─ Checking: amount (7000) <= 5000?
   └─ Result: False

5️⃣  Final decision
   ❌ DENIED - Action blocked
   └─ Reason: Payment amount exceeds $50.00 limit
   └─ Denial logged in audit trail


False

## 7. Key Takeaways

- 🎯 **Manifests** describe agent actions with full context
- 🛡️ **Policies** determine what's allowed or denied
- 🔐 **Deterministic** decisions - same input = same output
- 📊 **Audit trail** - every decision is logged
- ⚡ **Fast** - policy evaluation in milliseconds

## Next Steps

1. ✅ Check out `02_financial_compliance.ipynb` for real-world scenarios
2. ✅ Explore `06_policy_development.ipynb` to write custom policies
3. ✅ Read the full documentation in `README.md`